In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

def interpolate_group(group):
    group = group.set_index("Date").sort_index()
    group["Value"] = group["Value"].interpolate(method="time", limit_direction="both")
    
    # Si aún quedan NaNs (por falta de datos), eliminar el grupo
    if group["Value"].isna().any():
        return pd.DataFrame(columns=group.columns)  # grupo vacío
    return group.reset_index()


df = pd.read_csv("Muestra Tec Estudio de Precios Airbnb 2025_05_26 - 3 meses data.csv")  


columnas_metadata = [
    "Nombre", "#", "User", "Puntaje", "Profile", "Link",
    "Limpieza", "Veracidad", "Check-in", "Comunicación",
    "Ubicación", "Calidad", "Reviews", "Tarifa", "Cleaning",
    "Habs", "Jacuzzi", "Noches 05", "Noches 06", "Noches 07"
]

columnas_fecha = []
for col in [col for col in df.columns if col not in columnas_metadata]:

    datetime.strptime(col.strip(), "%d/%m/%Y")
    columnas_fecha.append(col)

columnas_metadata = [col for col in columnas_metadata if col in df.columns]

df["ID"] = df.index + 1  

df_metadata = df[["ID"] + columnas_metadata]

df_fechas = df[["ID"] + columnas_fecha].melt(
    id_vars="ID",
    var_name="Date",
    value_name="Value"
)




df_fechas["Date"] = pd.to_datetime(df_fechas["Date"], format="%d/%m/%Y")

df_fechas = df_fechas.sort_values(["ID", "Date"])

df_fechas = df_fechas.groupby("ID", group_keys=False).apply(interpolate_group)
print(df_fechas)


inicio = pd.to_datetime("2025-05-25")
fin = pd.to_datetime("2025-08-1")
df_fechas = df_fechas[(df_fechas["Date"] >= inicio) & (df_fechas["Date"] <= fin)]
df_fechas['Value'] = df_fechas['Value'].replace(-1, np.nan)
df_metadata["Min"] = df_fechas.groupby("ID")["Value"].min().reindex(df_metadata.index)
df_metadata["Max"] = df_fechas.groupby("ID")["Value"].max().reindex(df_metadata.index)
df_metadata["Mean"] = df_fechas.groupby("ID")["Value"].mean().reindex(df_metadata.index)
df_metadata["Var"] = df_fechas.groupby("ID")["Value"].var().reindex(df_metadata.index)





          Date  ID  Value
0   2025-05-26   1  122.5
1   2025-05-27   1  122.5
2   2025-05-28   1  122.5
3   2025-05-29   1  122.5
4   2025-05-30   1   -1.0
..         ...  ..    ...
235 2026-01-16  97  105.0
236 2026-01-17  97  105.0
237 2026-01-18  97  105.0
238 2026-01-19  97  105.0
239 2026-01-20  97  105.0

[23280 rows x 3 columns]


C:\Users\EdsonAlvarez\AppData\Local\Temp\ipykernel_31256\2330196017.py:51: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

C:\Users\EdsonAlvarez\AppData\Local\Temp\ipykernel_31256\2330196017.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\EdsonAlvarez\AppData\Local\Temp\ipykernel_31256\2330196017.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [123]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

df = pd.read_csv("Muestra Tec Estudio de Precios Airbnb 2025_05_26 - 3 meses data.csv")
df_interpolated = df.copy()

columnas_metadata = [
    "Nombre", "#", "User", "Puntaje", "Profile", "Link",
    "Limpieza", "Veracidad", "Check-in", "Comunicación",
    "Ubicación", "Calidad", "Reviews", "Tarifa", "Cleaning",
    "Habs", "Jacuzzi", "Noches 05", "Noches 06", "Noches 07"
]

limite_fecha = pd.to_datetime("31/07/2025", dayfirst=True)

def filtrar_columnas_fecha(df):
    fechas_columnas = pd.to_datetime(df.columns, errors='coerce', dayfirst=True)
    columnas_a_mantener = (fechas_columnas.isna()) | (fechas_columnas <= limite_fecha)
    return df.loc[:, columnas_a_mantener]

df = filtrar_columnas_fecha(df)
df_interpolated = filtrar_columnas_fecha(df_interpolated)

columnas_fecha_validas = [col for col in df_interpolated.columns if col not in columnas_metadata]
df_interpolated[columnas_fecha_validas] = df_interpolated[columnas_fecha_validas].replace(-1, np.nan)
df_interpolated[columnas_fecha_validas] = df_interpolated[columnas_fecha_validas].interpolate(axis=1, limit_direction='both')

columnas_fecha = []
for col in df.columns:
    if col not in columnas_metadata:
        try:
            datetime.strptime(col.strip(), "%d/%m/%Y")
            columnas_fecha.append(col)
        except:
            pass

columnas_metadata = [col for col in columnas_metadata if col in df.columns]

df["ID"] = df.index + 1
df_interpolated["ID"] = df_interpolated.index + 1

df_metadata = df[["ID"] + columnas_metadata]
df_metadata_inter = df_interpolated[["ID"] + columnas_metadata]

df_fechas = df[["ID"] + columnas_fecha].melt(
    id_vars="ID", var_name="Date", value_name="Value"
)
df_fechas_inter = df_interpolated[["ID"] + columnas_fecha].melt(
    id_vars="ID", var_name="Date", value_name="Value"
)

df_fechas["Date"] = pd.to_datetime(df_fechas["Date"], format="%d/%m/%Y")
df_fechas_inter["Date"] = pd.to_datetime(df_fechas_inter["Date"], format="%d/%m/%Y")

df_fechas = df_fechas.sort_values(["ID", "Date"])
df_fechas_inter = df_fechas_inter.sort_values(["ID", "Date"])

inicio = pd.to_datetime("2025-05-25")
fin = pd.to_datetime("2025-08-01")
df_fechas = df_fechas[(df_fechas["Date"] >= inicio) & (df_fechas["Date"] <= fin)]
df_fechas_inter = df_fechas_inter[(df_fechas_inter["Date"] >= inicio) & (df_fechas_inter["Date"] <= fin)]

df_fechas['Value'] = df_fechas['Value'].replace(-1, np.nan)

for metric in ["min", "max", "mean", "var"]:
    df_metadata[metric.capitalize()] = [df_fechas[df_fechas["ID"]==i]["Value"].agg(metric) for i in range(1,len(df_metadata)+1)]
    df_metadata_inter[metric.capitalize()] = [df_fechas_inter[df_fechas_inter["ID"]==i]["Value"].agg(metric) for i in range(1,len(df_metadata_inter)+1)]


conn = sqlite3.connect("airbnb_colombia.sqlite")
df_metadata.to_sql("Metadata", conn, if_exists="replace", index=False)
df_metadata_inter.to_sql("MetadataInterpolada", conn, if_exists="replace", index=False)
df_fechas.to_sql("Fechas", conn, if_exists="replace", index=False)
df_fechas_inter.to_sql("FechasInterpolada", conn, if_exists="replace", index=False)
conn.close()


C:\Users\EdsonAlvarez\AppData\Local\Temp\ipykernel_31256\791562876.py:19: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\EdsonAlvarez\AppData\Local\Temp\ipykernel_31256\791562876.py:19: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\EdsonAlvarez\AppData\Local\Temp\ipykernel_31256\791562876.py:68: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\EdsonAlvarez\AppData\Local\Temp\ipykernel_31256\791562876.py:69: SettingWithCopyWarning:


A value is tryin

In [124]:
df_metadata_inter.head()


,ID,Nombre,#,User,Puntaje,Profile,Link,Limpieza,Veracidad,Check-in,...,Cleaning,Habs,Jacuzzi,Noches 05,Noches 06,Noches 07,Min,Max,Mean,Var
0,1,Luxury |Boho-Chic |Jacuzzi| Dominique,NaN,Alejandra,5.0,NaN,NaN,5.0,5.0,5.0,...,$36.00,2,SI,15,25,31,113.0,122.5,115.772388,7.831323
1,2,14th Floor/Jacuzzi balcony/SeaViews/Parking/King,14XX,Alejandra,5.0,NaN,NaN,4.9,5.0,5.0,...,$38.00,2,SI,3,19,18,117.5,155.5,138.440299,136.770898
2,3,2 BR Modern APT jacuzzi / Rooftop near to beach,NaN,Alberto,5.0,NaN,NaN,5.0,5.0,5.0,...,$35.00,2,SI,10,31,30,111.0,117.0,113.776119,6.880936
3,4,Piso 12/ Jacuzzi/ Vistas/ 1BR/Parqueadero/Pisc...,NaN,Alejandra,5.0,NaN,NaN,5.0,5.0,5.0,...,$36.00,1,SI,20,24,31,87.5,112.0,100.634328,67.275622
4,5,Private jacuzzi |King bed |Parking| Wi-Fi,NaN,Alejandra,5.0,NaN,NaN,5.0,5.0,5.0,...,$36.00,2,SI,15,21,31,91.0,117.5,109.238806,50.961508
